

# Tạo dataset phân tích cảm xúc đơn giản




In [2]:
pip install pandas numpy # cài đặt thư viện

In [3]:
import pandas as pd
import numpy as np
import random

# 1. Định nghĩa lại các câu mẫu (Seed Sentences) với số lượng lớn hơn
positive_templates = [
    "The customer support was exceptionally helpful.",
    "I would gladly buy this item again.",
    "Five stars across the board for this experience.",
    "The delivery was fast and the product was in perfect condition.",
    "This gadget is a game-changer for my daily routine.",
    "The attention to detail in the packaging was impressive.",
    "I finally found what I was looking for, and it's superb.",
    "My family loved the outcome, highly satisfied.",
    "This brand truly cares about its customers.",
    "The installation process was surprisingly simple.",
    "The quality for the price is unbeatable.",
    "I've recommended this to all my friends already.",
    "A hidden gem of a service, I'm thrilled.",
    "They handled my request with speed and professionalism.",
    "This is the definition of a premium purchase.",
    "I am ecstatic about the performance of this machine.",
    "The design is sleek and modern, absolutely beautiful.",
    "This purchase brought me so much joy.",
    "Every expectation was met, and then some.",
    "I feel like I got an incredible deal.",
    "The interface is intuitive and very user-friendly.",
    "I can't find a single fault with this experience.",
    "What a reliable and durable item.",
    "I am definitely coming back for more purchases.",
    "This company sets a new standard for quality.",
    "The aroma and taste were simply divine.",
    "Their dedication to excellence is obvious.",
    "I am blown away by how effective this is.",
    "A flawless transaction from start to finish.",
    "The sound quality is crystal clear and rich.",
    "This little thing works wonders.",
    "I'm completely impressed with the durability.",
    "Best money I've spent all year.",
    "I had a great time using this feature.",
    "The vibrant colors are stunning.",
    "This is exactly what the description promised.",
    "A smooth and enjoyable process overall.",
    "The packaging was sturdy and secure.",
    "The staff went above and beyond for me.",
    "I give this my highest seal of approval."
]

negative_templates = [
    "The customer support was rude and totally unhelpful.",
    "I would never buy this item again, terrible quality.",
    "Zero stars if I could for this horrible experience.",
    "The delivery was delayed and the product arrived damaged.",
    "This gadget broke after only one week of use.",
    "The packaging was cheap and the item was scratched.",
    "I wasted my time and money on this useless product.",
    "My friends laughed at the poor outcome.",
    "This brand clearly doesn't care about its customers.",
    "The installation instructions were completely confusing.",
    "The quality for the price is a total rip-off.",
    "I've warned all my friends not to buy this.",
    "A huge disappointment of a service, I'm angry.",
    "They handled my request slowly and with utter incompetence.",
    "This is the definition of a regretful purchase.",
    "I am furious about the terrible performance of this machine.",
    "The design is ugly and feels cheap.",
    "This purchase caused me a lot of stress.",
    "None of my expectations were remotely met.",
    "I feel like I was totally scammed.",
    "The interface is confusing and very difficult to navigate.",
    "I can find countless faults with this terrible item.",
    "What an unreliable and flimsy product.",
    "I am definitely avoiding this company forever.",
    "This company has a low standard for quality.",
    "The food tasted bland and stale.",
    "Their lack of professionalism is alarming.",
    "I am shocked by how ineffective this is.",
    "A messy and frustrating transaction from start to finish.",
    "The sound quality is distorted and muffled.",
    "This little thing doesn't work at all.",
    "I'm utterly unimpressed with the fragility.",
    "Worst money I've spent this month.",
    "I had a horrible experience trying to use this feature.",
    "The faded colors are dull.",
    "This is nothing like the description promised.",
    "A bumpy and stressful process overall.",
    "The packaging was flimsy and the item was loose.",
    "The staff was lazy and unmotivated.",
    "I give this my lowest rating possible."
]

# 2. Định nghĩa các từ đồng nghĩa/thay thế được mở rộng
modifiers = {
    # General Adjectives/Adverbs
    "is": ["is", "was", "is truly", "is absolutely", "is simply", "turned out to be", "feels", "seemed"],
    "absolutely": ["absolutely", "totally", "utterly", "completely", "genuinely", "really", "so"],
    "very": ["very", "extremely", "incredibly", "highly", "so", "super"],
    "excellent": ["excellent", "superb", "outstanding", "fantastic", "wonderful", "great", "top-notch", "amazing"],
    "terrible": ["terrible", "horrible", "awful", "dreadful", "lousy", "poor", "unpleasant", "bad"],

    # Nouns (Product/Service/Experience)
    "product": ["product", "item", "gadget", "machine", "device", "thing", "app", "model", "unit"],
    "service": ["service", "support", "assistance", "help", "staff", "team", "care"],
    "experience": ["experience", "transaction", "purchase", "interaction", "visit", "deal", "order"],
    "quality": ["quality", "material", "finish", "build", "construction", "standard"],
    "design": ["design", "look", "appearance", "style", "interface", "layout"],

    # Verbs/Phrases
    "buy this item again": ["buy this item again", "reorder this", "recommend this", "come back", "make another purchase"],
    "was fast": ["was fast", "was quick", "was speedy", "arrived quickly", "was prompt"],
    "was slow": ["was slow", "was delayed", "was sluggish", "took forever", "was late"],
    "broke": ["broke", "malfunctioned", "stopped working", "failed", "died", "gave up"],
    "unhelpful": ["unhelpful", "useless", "incompetent", "rude", "unprofessional", "clueless"],
    "recommended": ["recommended", "suggested", "mentioned", "told others about", "vouched for"],
    "satisfied": ["satisfied", "happy", "pleased", "content", "thrilled", "joyful", "delighted"],
    "disappointed": ["disappointed", "angry", "furious", "upset", "frustrated", "let down", "annoyed"],
    "met my expectations": ["met my expectations", "lived up to the hype", "delivered as promised", "achieved the goal"],
    "waste of money": ["waste of money", "rip-off", "bad deal", "poor value", "financial blunder"],
    "works": ["works", "operates", "functions", "runs", "performs"],
    "flawlessly": ["flawlessly", "perfectly", "without a hitch", "smoothly", "easily", "beautifully"],
    "unacceptable": ["unacceptable", "intolerable", "outrageous", "disgraceful", "shameful"],
    "simple": ["simple", "easy", "straightforward", "intuitive", "hassle-free", "quick"],
    "loved": ["loved", "adored", "enjoyed", "cherished"],
    "hated": ["hated", "detested", "despised", "loathed"],
    "purchase": ["purchase", "buy", "decision", "investment", "order"]
}

def generate_sentence(template, label):
    """Tạo ra một câu ngẫu nhiên dựa trên template và modifiers."""
    sentence = template
    for original, alternatives in modifiers.items():
        if original in sentence:
            # Ngẫu nhiên thay thế từ gốc bằng một từ đồng nghĩa
            if random.random() < 0.7:  # 70% cơ hội thay thế
                sentence = sentence.replace(original, random.choice(alternatives), 1)

    # Thêm một chút ngẫu nhiên về cấu trúc câu
    if label == "Positive" and random.random() < 0.2:
        sentence = "Seriously, " + sentence
    elif label == "Negative" and random.random() < 0.2:
        sentence = sentence + ". Stay away."

    return sentence

# 3. Tạo dữ liệu 10.000 mẫu
NUM_SAMPLES = 10000
data = []

# Chia đều 5000 Positive và 5000 Negative
for i in range(NUM_SAMPLES):
    if i < NUM_SAMPLES // 2:
        label = "Positive"
        template = random.choice(positive_templates)
    else:
        label = "Negative"
        template = random.choice(negative_templates)

    # Tạo câu ngẫu nhiên
    text = generate_sentence(template, label)

    # Thêm vào danh sách dữ liệu
    data.append({
        "ID": i + 1,
        "Text": text,
        "Label": label
    })

# 4. Tạo DataFrame và Xuất file CSV
df = pd.DataFrame(data)

# Đảo thứ tự các hàng ngẫu nhiên (trộn lẫn Positive và Negative)
df = df.sample(frac=1).reset_index(drop=True)

# Cập nhật lại cột ID sau khi trộn
df['ID'] = df.index + 1

# Xuất ra file CSV mới
file_name = "sentiment_dataset_10000_v2.csv"
df.to_csv(file_name, index=False)

print(f"✅ Đã tạo thành công dataset 10.000 mẫu đa dạng hơn.")
print(f"Tên file: {file_name}")
print(f"Cấu trúc 5 hàng đầu tiên:")
print(df.head())

✅ Đã tạo thành công dataset 10.000 mẫu đa dạng hơn.
Tên file: sentiment_dataset_10000_v2.csv
Cấu trúc 5 hàng đầu tiên:
   ID                                               Text     Label
0   1             Thwas decision brought me so much joy.  Positive
1   2  This gadget is a game-changer for my daily rou...  Positive
2   3           This so purchase brought me so much joy.  Positive
3   4  Thseemed company sets a new standard for quality.  Positive
4   5    The staff was lazy and unmotivated.. Stay away.  Negative
